In [1]:
import numpy as np
#import sys, os
#sys.path.append(os.path.abspath('../../src/neural_network'))
#sys.path.append('../../src/neural_network')
from neural_network import NeuralNetwork
from fc_layer import FCLayer
from activation_layer import ActivationLayer
from activation_functions import tanh, tanh_derivative, relu, relu_derivative
from loss_functions import mse, mse_derivative
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
import sklearn
from sklearn import *

In [2]:
X, Y = datasets.load_diabetes(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(442, 10) (442,)
[151. 317. 141. 131. 127. 263.  66. 257. 274.  37. 292. 166. 167. 173.
 104. 272. 134. 108.  81.  88. 109. 264. 150. 237.  93. 310.  75.  65.
  68.  64. 259. 104. 214. 232. 259. 103. 136. 152. 277. 177. 124. 219.
 265. 107. 134. 236.  52.  47. 178.  68. 125. 122.  39. 163. 170.  96.
 122.  90. 143. 235.  45.  59. 111. 281. 182.  67. 198.  60.  70. 243.
 214. 101.  52. 268.  83. 102.  96. 116.  84. 168. 220. 171. 242.  77.
 249.  72. 275. 180. 143.]
(353, 1, 10) (89, 1, 10) (353, 1, 1) (89,)


In [6]:
network = NeuralNetwork()
network.add(FCLayer(X_train.shape[2], 5))
network.add(ActivationLayer(relu, relu_derivative))
network.add(FCLayer(5, 2))
network.add(ActivationLayer(relu, relu_derivative))
network.add(FCLayer(2, 1))
network.add(ActivationLayer(relu, relu_derivative))
network.use(mse, mse_derivative)
network.fit(X_train, y_train, epochs=100, learning_rate=0.001)
y_pred = np.array(network.predict(X_test))
y_pred = y_pred.reshape(y_pred.shape[0],)
print(y_test, y_test.shape)
print(y_pred, y_pred.shape)
print(sklearn.metrics.mean_squared_error(y_test, y_pred))

[151. 317. 141. 131. 127. 263.  66. 257. 274.  37. 292. 166. 167. 173.
 104. 272. 134. 108.  81.  88. 109. 264. 150. 237.  93. 310.  75.  65.
  68.  64. 259. 104. 214. 232. 259. 103. 136. 152. 277. 177. 124. 219.
 265. 107. 134. 236.  52.  47. 178.  68. 125. 122.  39. 163. 170.  96.
 122.  90. 143. 235.  45.  59. 111. 281. 182.  67. 198.  60.  70. 243.
 214. 101.  52. 268.  83. 102.  96. 116.  84. 168. 220. 171. 242.  77.
 249.  72. 275. 180. 143.] (89,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (89,)
29440.977528089887


In [3]:
def my_func(config, reporter):  # add the reporter parameter
  # report the configuration
  network = NeuralNetwork()
  network.add(FCLayer(config['input_dim'], config['hidden_dim']))
  network.add(ActivationLayer(relu, relu_derivative))
  network.add(FCLayer(config['hidden_dim'], 2))
  network.add(ActivationLayer(relu, relu_derivative))
  network.add(FCLayer(2, config['output_dim']))
  network.add(ActivationLayer(relu, relu_derivative))
  network.use(mse, mse_derivative)
  network.fit(X_train, y_train, epochs=100, learning_rate=config["alpha"])
  y_pred = np.array(network.predict(X_test))
  y_pred = y_pred.reshape(y_pred.shape[0],)
  reporter(config, mean_loss = sklearn.metrics.mean_squared_error(y_test, y_pred))

In [5]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
        my_func,
        name="my_exp",
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=10,
        config={
            "steps": 100,
            "input_dim": X_train.shape[2],
            "hidden_dim": 20,
            
            "output_dim": 1,
            "alpha": tune.uniform(0.0001, 10),
        },
    )

print("Best hyperparameters found were: ", analysis.best_config)

2022-04-14 06:29:07,431	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,alpha
my_func_6c6a401d,RUNNING,,3.74546


2022-04-14 06:29:11,631	WARNING tune.py:518 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,alpha
my_func_6c6a401d,RUNNING,,3.74546
my_func_6c7f2ec1,RUNNING,,9.50715
my_func_6c916212,PENDING,,7.31997


2022-04-14 06:29:11,880	ERROR tune.py:557 -- Trials did not complete: [my_func_6c6a401d, my_func_6c7f2ec1, my_func_6c916212]
2022-04-14 06:29:11,881	INFO tune.py:561 -- Total run time: 4.46 seconds (4.20 seconds for the tuning loop).
2022-04-14 06:29:11,882	WARNING tune.py:565 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
2022-04-14 06:29:11,897	WARNING experiment_analysis.py:644 -- Could not find best trial. Did you pass the correct `metric` parameter?


Best hyperparameters found were:  None
